In [1]:
import os 
os.environ['CUDA_VISIBLE_DEVICES']='7'
import keras 
from keras.layers.merge import Grad 
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger


Using TensorFlow backend.


In [ ]:
#load the data 
import h5py 
import numpy as np 
data_prefix="/srv/scratch/annashch/deeplearning/gradient_exp/"
train_hdf5=h5py.File(data_prefix+"train_data.hdf5",'r')
train_X=train_hdf5['X']['sequence']
train_Y=train_hdf5['Y']['output']

valid_hdf5=h5py.File(data_prefix+'valid_data.hdf5','r')
valid_X=valid_hdf5['X']['sequence']
valid_Y=valid_hdf5['Y']['output']

test_hdf5=h5py.File(data_prefix+"test_data.hdf5",'r')
test_X=test_hdf5['X']['sequence']
test_Y=test_hdf5['Y']['output']

weights=np.load('weights.npy')
Y_grads=np.load('gradients.npy')

In [ ]:
#build the model -- without gradient reward (this is our baseline)
input_shape=(200,4)
pool_size=10 
inp = keras.layers.Input(shape=input_shape)
conv = keras.layers.Convolution1D(nb_filter=15, filter_length=15, subsample_length=1)(inp)
relu_post_conv = keras.layers.Activation("relu")(conv)
gap = keras.layers.pooling.GlobalAveragePooling1D()(relu_post_conv)
dense = keras.layers.Dense(2)(gap)
sigmoid_out = keras.layers.Activation("sigmoid")(dense)
model = keras.models.Model(input=inp, output=sigmoid_out)
model.compile(optimizer="adam", loss="binary_crossentropy",metrics=["binary_accuracy"])

In [ ]:
#train the model 
model_output_path="benchmarkModel"
#checkpointer = ModelCheckpoint(filepath=model_output_path)
earlystopper = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
tensorboard = TensorBoard( write_images=True)
csv_logger = CSVLogger('training.log')
model.fit(x=np.asarray(train_X), 
          y=np.asarray(train_Y),
          callbacks=[earlystopper,csv_logger,tensorboard],
          batch_size=250,
          epochs=50,
          validation_data=tuple([valid_X,valid_Y]),
          verbose=0)

In [4]:
#build the model with gradient reward 
reload(keras.layers.merge)
from keras.layers.merge import Grad
input_shape=(200,4)
pool_size=10 
inp = keras.layers.Input(shape=input_shape)
conv = keras.layers.Convolution1D(nb_filter=15, filter_length=15, subsample_length=1)(inp)
relu_post_conv = keras.layers.Activation("relu")(conv)
gap = keras.layers.pooling.GlobalAveragePooling1D()(relu_post_conv)
dense = keras.layers.Dense(2)(gap)
sigmoid_out = keras.layers.Activation("sigmoid")(dense)
grad_layer1 = Grad(task_index=0)([inp, dense])
grad_layer2 = Grad(task_index=1)([inp, dense])
model = keras.models.Model(input=inp, output=[sigmoid_out, grad_layer])
model.compile(sample_weight_mode=[None,"temporal"],optimizer="adam", loss=["binary_crossentropy",keras.losses.get_positionwise_cosine_1d(pool_size=pool_size)],metrics=["binary_accuracy"])

/users/annashch/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(strides=1, filters=15, kernel_size=15)`


> /users/annashch/keras/keras/layers/merge.py(211)_merge_function()
-> the_inp=inputs[0]
(Pdb) id(inputs[0])
139999098911120
(Pdb) id(inputs[1])
139999098661456
(Pdb) c
> /users/annashch/keras/keras/layers/merge.py(211)_merge_function()
-> the_inp=inputs[0]
(Pdb) id(inp[0])
*** NameError: name 'inp' is not defined
(Pdb) id(inputs[0])
139999098911120
(Pdb) id(inputs[1])
139999098429520
(Pdb) c


ValueError: slice index 1 of dimension 1 out of bounds. for 'grad_4/strided_slice' (op: 'StridedSlice') with input shapes: [?,1,2], [2], [2], [2] and with computed input tensors: input[1] = <0 1>, input[2] = <0 2>, input[3] = <1 1>.